The goal of this notebook is to implement a trie structure to quickly find all available uuidV4 listings.The script responsible for maintaining the listings collection needs the UUID to be searchabel in a quick manner. This notebook is intended to implement a trie class as well as perform a benchmark to compare the trie-performance vs the standard python`in ` operator.

In [1]:
import sys
sys.path.append('../utils')
import config_handling as conf
from database import Database
from trie import UUIDv4Trie
import time
import uuid


In [2]:
# Connect to database
config = conf.read_config('../config/automotive.conf.ini')
config.read('config.ini')
connection_type = config['settings']['connection']
connection_type
user = config[connection_type]['user']
pw = config[connection_type]['pw']
host = config[connection_type]['host']
db = config[connection_type]['db']
port = config[connection_type].getint('port')
db = Database(host,
              port,
              user,
              pw,
              db
              )
db.connect()
db.start_transaction()
#image directory: 
basedir = config['settings']['image_directory']

Connection established


In [3]:
trie = UUIDv4Trie()

query  = "SELECT autoscout_id FROM listings"
rows = db.execute_query(query)
for row in rows:
    #I'm not sure if Python optimizes things if the same loop occurs a second time
    #loop here onces, then loop again and again to limit the potential impact of this.
    id = row['autoscout_id']

trie_build_starttime = time.time()
for row in rows:
    id = row['autoscout_id']
    trie.insert(id)
trie_build_time = time.time() - trie_build_starttime

inlist = []
inlist_build_starttime = time.time()
for row in rows:
    #todo; is there a faster way?
    id = row['autoscout_id']
    inlist.append(id)
inlist_build_time = time.time() - inlist_build_starttime

print(f"Execution times: \n \t Trie: {trie_build_time}\n\t List: {inlist_build_time}")

Execution times: 
 	 Trie: 52.92005109786987
	 List: 0.17224574089050293


So we have establisted that making a trie-structure for our data is a lot slower than using a list. What about using the in operator vs trie-traversal (i.e. lookup if uuid exists)?

In [4]:
import numpy as np
import random

In [5]:
numpy_based_ids = np.array(inlist)

In [6]:
random_uuids = np.random.choice(numpy_based_ids, 1000)
#add 250 random uuids:
for i in range(250):
    uuidstr = str(uuid.uuid4())
    random_uuids = np.append(random_uuids, uuidstr)
random.shuffle(random_uuids)
len(random_uuids)

1250

In [7]:
nparr = np.array(inlist)


In [8]:
for id in random_uuids:
    #same thing; exclude cashing issue.
    pass
#measure in-operator in python list: 
in_loop_start_time = time.time()
yes = 0
no = 0
for id in random_uuids:
    if id in inlist:
        yes += 1
    else:
        no += 1
in_loop_duration = time.time() - in_loop_start_time
print(f"Time taken by in-operator (list): {in_loop_duration} with {yes}yesses and {no} nos")
#measure in-operator in numpy array: (aborted during testing, stupidly slow)
#yes = 0
#no = 0
#nparr_start_time = time.time()
#for id in random_uuids:
#    if np.isin(id, nparr):
#        yes += 1
#    else:
#        no += 1
#npar_duration = time.time() - nparr_start_time
#print(f"Time taken by in-operator (NP.array): {npar_duration } with {yes}yesses and {no} nos")
#trie structure: 
yes = 0
no = 0
trie_starttime = time.time()
for id in random_uuids:
    if trie.search(id):
        yes += 1
    else:
        no += 1
trie_duration = time.time()  - trie_starttime
print(f"Time taken by trie-searcher: {trie_duration } with {yes}yesses and {no} nos")




Time taken by in-operator (list): 19.924341440200806 with 1000yesses and 250 nos
Time taken by trie-searcher: 0.008815288543701172 with 1000yesses and 250 nos


WOOHOO: building trie is a lot slower, but seems to be worth it! 
Conclusion: Trie datastructure should be implemented in the maintainer script!